In [1]:
# Import Tecton and other libraries
import logging
import os
import tecton
from dotenv import load_dotenv
import pandas as pd
import snowflake.connector
from datetime import datetime, timedelta
from pprint import pprint

load_dotenv()  # take environment variables from .env.
logging.getLogger('snowflake.connector').setLevel(logging.WARNING)
logging.getLogger('snowflake.snowpark').setLevel(logging.WARNING)

connection_parameters = {
    "user": INSERT_USER,
    "password": INSERT_PASSWORD,
    "account": "tectonpartner",
    "warehouse": "DEMO_WH",
    # Database and schema are required to create various temporary objects by tecton
    "database": "SANIKA",
    "schema": "PUBLIC",
}
conn = snowflake.connector.connect(**connection_parameters)
tecton.snowflake_context.set_connection(conn) # Tecton will use this Snowflake connection for all interactive queries


# Quick helper function to query snowflake from a notebook
# Make sure to replace with the appropriate connection details for your own account
def query_snowflake(query):
    df = conn.cursor().execute(query).fetch_pandas_all()
    return df

tecton.version.summary()

Version: 0.5.0b27
Git Commit: 88fa4a7397fb3e92d32bfbe073077487ed71ded6
Build Datetime: 2022-07-20T18:51:35


In [2]:
ws = tecton.get_workspace('sanika-snowflake')

In [3]:
fv = ws.get_feature_view('user_transaction_metrics')

In [4]:
start_time = datetime.utcnow()-timedelta(days=30)
end_time = datetime.utcnow()

In [14]:
fv.run(start_time=yesterday, end_time=today).to_pandas().head()

,USER_ID,TIMESTAMP,INT_VALUE_VARIANCE_1D_1D,INT_VALUE_STDDEV_1D_1D
0,user_a,2022-05-15,1.583333,1.258306
1,user_b,2022-05-15,NaN,NaN
2,user_a,2022-05-16,3.333333,1.825742
3,user_b,2022-05-16,NaN,NaN
4,user_a,2022-05-17,3.333333,1.825742


In [10]:
fv.get_historical_features(start_time=yesterday, end_time=today).to_pandas().head()

,USER_ID,TIMESTAMP,INT_VALUE_VARIANCE_1D_1D,INT_VALUE_STDDEV_1D_1D
0,user_a,2022-05-15,1.583333,1.258306
1,user_b,2022-05-15,NaN,NaN
2,user_a,2022-05-16,3.333333,1.825742
3,user_b,2022-05-16,NaN,NaN
4,user_a,2022-05-17,3.333333,1.825742


In [9]:
tecton.conf.set("TECTON_API_KEY", "<key>")

keys = {
    'USER_ID': 'user_a',
}
features = fs.get_online_features(join_keys=keys).to_dict()
pprint(features)

curl -X POST --silent https://<your-cluster>.tecton.ai/api/v1/feature-service/get-features\
     -H "Authorization: Tecton-key $TECTON_API_KEY" -d\
'{
  "params": {
    "feature_service_name": "fraud_detection_feature_service",
    "join_key_map": {
      "USER_ID": "user_461615966685",
      "CATEGORY": "grocery_net"
    },
    "workspace_name": "prod"
  }
}' | jq